# Testing notebook

In [1]:
import xlsxwriter
import pandas as pd
import numpy as np

from classification_power_predictor import classification_power_predictor
from writing import *

## We need some testing data

In [2]:
# frame size
n = 1000

test_frame = pd.DataFrame({"numeric_variable": np.random.normal(10, 5, n), 
                           "object_variable": np.round(np.random.uniform(0, 10, n)).astype('O'),
                           "column with soo000oo000oo000oo000oo long name": np.random.normal(10, 5, n)})

nv = test_frame["numeric_variable"]
ov = test_frame["object_variable"]

probs1 = nv.apply(lambda x: (x/(max(nv) - min(nv))) + ((np.random.rand())/5))
probs2 = ov.apply(lambda x: (x/(max(ov) - min(ov))) + ((np.random.rand())/5))

f_probs = (probs1 + probs2)/2
f_probs[f_probs >= 1] = 1


Y = np.zeros(n)
Y[f_probs > 0.4] = 1
Y[f_probs > 0.7] = 2

Y = pd.Series(Y)
Y_strs = Y.replace({0: 'cetegory 0', 1:'category 1', 2:'category 2'})
Y.value_counts()
Y_binary = Y.replace({2:1})

na_containts_frame = test_frame.copy()

for col in test_frame.columns:
    na_containts_frame.loc[na_containts_frame.sample(int(n/50)).index, col] = np.NaN

## Computions funcitons

In [3]:
from computions import *

### get_describe_nominal function test

Basic situation

In [4]:
get_describe_nominal(test_frame.iloc[:,1], Y)

object_variable


,count,0.0,0.0%,1.0,1.0%,2.0,2.0%
object_variable,,,,,,,
1.0,113,106.0,93.805310,7.0,6.194690,0.0,0.000000
5.0,110,19.0,17.272727,90.0,81.818182,1.0,0.909091
6.0,104,6.0,5.769231,98.0,94.230769,0.0,0.000000
2.0,103,87.0,84.466019,16.0,15.533981,0.0,0.000000
8.0,102,0.0,0.000000,77.0,75.490196,25.0,24.509804
3.0,95,60.0,63.157895,35.0,36.842105,0.0,0.000000
7.0,92,0.0,0.000000,87.0,94.565217,5.0,5.434783
9.0,91,0.0,0.000000,60.0,65.934066,31.0,34.065934
4.0,89,36.0,40.449438,53.0,59.550562,0.0,0.000000


Data frame with emptys

In [5]:
test = get_describe_nominal(na_containts_frame.iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))
test = get_describe_nominal(na_containts_frame.fillna('empty').iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))

object_variable
Is all obs. are in table -  False
object_variable
Is all obs. are in table -  True


### AUC computing test

AUC numeric - simple fucntion

In [6]:
print(get_AUC_numeric(test_frame['numeric_variable'] ,Y))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(na_containts_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_binary))

{0.0: 0.3216474898979522, 1.0: 0.5754920420347929, 2.0: 0.7796336996336996}
{'cetegory 0': 0.3216474898979522, 'category 1': 0.5754920420347929, 'category 2': 0.7796336996336996}
{'cetegory 0': 0.32634814567323756, 'category 1': 0.569800319187705, 'category 2': 0.784307480635956}
{0.0: 0.3216474898979522, 1.0: 0.6783525101020479}


AUC nominal

In [7]:
non_table = get_AUC_nominal(test_frame['object_variable'], Y)
dn_tab = get_describe_nominal(test_frame.iloc[:,1], Y_strs)
with_table = get_AUC_nominal(test_frame['object_variable'], Y_strs, dn_tab)
print(with_table)

print('AUC bynary')
print(get_AUC_nominal(test_frame['object_variable'], Y_binary))

print('AUC with emptys - needs to be replaces in previous steps')
print(get_AUC_nominal(na_containts_frame['object_variable'].fillna('empty'), Y_strs))

object_variable
object_variable
{'cetegory 0': 0.9502281521813574, 'category 1': 0.8755411255411255, 'category 2': 0.9302380952380953}
AUC bynary
object_variable
{0.0: 0.9502281521813574, 1.0: 0.9502281521813574}
AUC with emptys - needs to be replaces in previous steps
object_variable
{'cetegory 0': 0.9479423498390521, 'category 1': 0.8727852073576979, 'category 2': 0.927954822954823}


Get publish AUC for any predictor type

In [8]:
non_table = get_full_AUC(test_frame['object_variable'], Y, 'nominal')
print('nominal predictor without table')
print(non_table)

with_table = get_full_AUC(test_frame['object_variable'], Y_strs, 'nominal', dn_tab)
print('moninal predictor with table')
print(with_table)

with_nas = get_full_AUC(na_containts_frame['object_variable'], Y, 'nominal', fillna_nominal = 'empty' )
print('nominal predictor with empty values')
print(with_nas)

numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
print('numeric predictor')
print(numeric_var)

object_variable
nominal predictor without table
{0.0: {'AUC': 0.9502281521813574, 'rel_type': 1, 'GINI': 0.9004563043627147}, 1.0: {'AUC': 0.8755411255411255, 'rel_type': 1, 'GINI': 0.751082251082251}, 2.0: {'AUC': 0.9302380952380953, 'rel_type': 1, 'GINI': 0.8604761904761906}}
moninal predictor with table
{'cetegory 0': {'AUC': 0.9502281521813574, 'rel_type': 1, 'GINI': 0.9004563043627147}, 'category 1': {'AUC': 0.8755411255411255, 'rel_type': 1, 'GINI': 0.751082251082251}, 'category 2': {'AUC': 0.9302380952380953, 'rel_type': 1, 'GINI': 0.8604761904761906}}
object_variable
nominal predictor with empty values
{0.0: {'AUC': 0.9479423498390521, 'rel_type': 1, 'GINI': 0.8958846996781042}, 1.0: {'AUC': 0.8727852073576979, 'rel_type': 1, 'GINI': 0.7455704147153959}, 2.0: {'AUC': 0.927954822954823, 'rel_type': 1, 'GINI': 0.855909645909646}}
numeric predictor
{0.0: {'AUC': 0.6783525101020478, 'rel_type': -1, 'GINI': 0.35670502020409556}, 1.0: {'AUC': 0.5754920420347929, 'rel_type': 1, 'GINI'

Decoding AUC info as DataFrame

In [9]:
numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
AUC_info_to_DataFrame(numeric_var,'numeric_variable')

0.0                          1.0                     \
                       AUC rel_type      GINI       AUC rel_type      GINI   
numeric_variable  0.678353       -1  0.356705  0.575492        1  0.150984   

                       2.0                     
                       AUC rel_type      GINI  
numeric_variable  0.779634        1  0.559267

Getting indicators from computions uotput

In [10]:
comp_result =  get_all_comuptions(test_frame['numeric_variable'], Y)
print('full numeric data')
get_predictor_row(comp_result)
print('numeric data with nas, with str categoryes')
comp_result =  get_all_comuptions(na_containts_frame['numeric_variable'], Y_strs)
get_predictor_row(comp_result)

full numeric data
numeric data with nas, with str categoryes


cetegory 0                    category 1                     \
                        AUC rel_type      GINI        AUC rel_type      GINI   
numeric_variable   0.673652       -1  0.347304     0.5698        1  0.139601   

                 category 2                    Emptys Emptys part  
                        AUC rel_type      GINI                     
numeric_variable   0.784307        1  0.568615     20        0.02

## Writing to excel

Adding a new sheet

In [11]:
xl_writer = pd.ExcelWriter("test_result/sheet_creation.xlsx",engine='xlsxwriter')

for i in range(20):
    add_sheet(xl_writer, 'some name')
    add_sheet(xl_writer, 'some very very very long long name')
xl_writer.close()

Adding a info about different predictors

In [12]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()

object_variable


Let's show final table

In [13]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
#my_cpp.get_predictors_data()
my_cpp.result_DF

object_variable


0.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.678353       -1  0.356705   
object_variable                                0.950228        1  0.900456   
column with soo000oo000oo000oo000oo long name  0.502247        1  0.004495   

                                                    1.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.575492        1  0.150984   
object_variable                                0.875541        1  0.751082   
column with soo000oo000oo000oo000oo long name  0.506268       -1  0.012536   

                                                    2.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.779634        1  0.559267   
object_variable                                0.930238        1  0.860476   
column with soo000oo000oo000oo000oo long name  0.512613        1  0.025226   

                                              Emptys Emptys part  
                                                                  
numeric_variable                                   0         0.0  
object_variable                                    0         0.0  
column with soo000oo000oo000oo000oo long name      0         0.0

Writig a double header table

In [14]:


xl_writer = pd.ExcelWriter("test_result/double_header_saver.xlsx",engine='xlsxwriter')
ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws      

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

print_double_column_header(  my_cpp, ws, 
                            my_cpp.result_DF.loc[['numeric_variable'],:],
                            "A1", 'Hello World', 
                            xl_writer.book.add_format(my_cpp.default_header_format))

xl_writer.close()

object_variable


Writing info about some predictor

In [15]:
xl_writer = pd.ExcelWriter("test_result/writing_a_value.xlsx",engine='xlsxwriter')

ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws                             

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

default_predictor_printer(my_cpp, ws, my_cpp.get_predictors_data()['numeric_variable'], 
{'header_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'desc_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'class_ab_format':xl_writer.book.add_format(my_cpp.default_header_format)})

xl_writer.close()

object_variable


In [17]:
xl_writer = pd.ExcelWriter("test_result/result_test.xlsx",engine='xlsxwriter')
my_cpp.write_to_book(xl_writer)
xl_writer.close()